In [1]:
from datasets import load_dataset
from utils import jpeg_compress, webp_compress, pad, crop, nn_compress, hific_lo_compress
import compressai
from evaluate import evaluator
from transformers import pipeline
import torch
import numpy as np
import matplotlib.pyplot as plt
from piq import LPIPS
from torch.nn import MSELoss

2023-10-25 22:49:31.852500: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-25 22:49:31.852548: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-25 22:49:31.852594: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [2]:
def jpeg_compress_beans(sample):
    img,bpp = jpeg_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample
    
def webp_compress_beans(sample):
    img,bpp = webp_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

net_mbt2018 = compressai.zoo.mbt2018(quality=1, pretrained=True).eval().to("cpu")
def mbt2018_compress_beans(sample):
    img,bpp = nn_compress(sample['image'],net_mbt2018,"cpu")
    sample['image'] = img
    sample['bpp'] = bpp
    return sample

def hific_lo_compress_beans(sample):
    img,bpp = hific_lo_compress(sample['image'])
    sample['image'] = img
    sample['bpp' ] = bpp
    return sample

In [3]:
image_compression_methods = [
    jpeg_compress_beans,
    webp_compress_beans,
    mbt2018_compress_beans,
    hific_lo_compress_beans
]

In [4]:
beans = [load_dataset("beans", split="validation[0:3]")]

In [9]:
for method in image_compression_methods:
    beans.append(beans[0].map(method))

Map:   0%|          | 0/3 [00:00<?, ? examples/s]

In [10]:
lpips_metric = LPIPS()
mse_metric = MSELoss()

/home/server/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/server/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG16_Weights.IMAGENET1K_V1`. You can also use `weights=VGG16_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [20]:
%%time
lpips = []
mse = []
for method in beans:
    method = method.with_format("torch")
    lpips.append([])
    mse.append([])
    for i_sample,sample in enumerate(beans[0].with_format("torch")):
        compressed_sample = method[i_sample]
        reference = sample['image'].to(torch.float32).permute(2,0,1).unsqueeze(0)
        distorted = compressed_sample['image'].to(torch.float32).permute(2,0,1).unsqueeze(0)
        lpips[-1].append(lpips_metric(reference,distorted).detach().item())
        mse[-1].append(mse_metric(reference,distorted).detach().item())

CPU times: user 3min 2s, sys: 12.9 s, total: 3min 14s
Wall time: 41 s


In [33]:
PSNR = [[20*np.log10(255)-10*np.log10(d) for d in m] for m in mse]
PSNR = [np.mean(d) for d in PSNR]
PSNR

/tmp/ipykernel_395408/986530016.py:1: RuntimeWarning: divide by zero encountered in log10
  PSNR = [[20*np.log10(255)-10*np.log10(d) for d in m] for m in mse]


[inf,
 20.763526877858002,
 21.96498775369172,
 22.925198120399873,
 21.829516076497146]

In [34]:
neg_log_lpips = [[-10*np.log10(d) for d in m] for m in lpips]
neg_log_lpips = [np.mean(d) for d in neg_log_lpips]
neg_log_lpips

/tmp/ipykernel_395408/1126017225.py:1: RuntimeWarning: divide by zero encountered in log10
  neg_log_lpips = [[-10*np.log10(d) for d in m] for m in lpips]


[inf,
 5.716154228423233,
 6.7499197306218255,
 6.899412762783137,
 9.779208600431419]